In [ ]:
pip install requests

# Imports

In [ ]:
import requests
import os
import pandas as pd
from bs4    import BeautifulSoup

# Mount Google Drive 

In [ ]:
currentWorkingDir = !pwd
defaultWorkingDir = "/content"

if ( currentWorkingDir[0] == defaultWorkingDir ):
  from google.colab import drive

  drive.mount('/content/drive')
      
  %cd "/content/drive/My Drive/Colab Notebooks/stock_portfolio"
else:
  print("Currenting running app from: ")
  !pwd


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/stock_portfolio


# Extract stock discloure information posted between 2016-2020

## Get list of entities

In [ ]:
df_listed     = pd.read_csv("listed-companies-main.csv")
df_listed_jnr = pd.read_csv("listed-companies-jnr.csv")

In [ ]:
df_listed

,Name,InstrumentCode,Currency,Sector,Type,Website
0,138 STUDENT LIVING JAMAICA LIMITED,138SL,JMD,OTHER,ORDINARY,-
1,138 STUDENT LIVING JAMAICA LIMITED VARIABLE PR...,138SLVR,JMD,OTHER,PREFERENCE,-
2,1834 INVESTMENTS LIMITED,1834,JMD,COMMUNICATIONS,ORDINARY,Website
3,BARITA INVESTMENTS LIMITED,BIL,JMD,FINANCE,ORDINARY,Website
4,BERGER PAINTS JAMAICA LTD.,BRG,JMD,MANUFACTURING,ORDINARY,Website
...,...,...,...,...,...,...
63,TRANSJAMAICAN HIGHWAY LIMITED,TJH,JMD,NaN,ORDINARY,-
64,TRANSJAMAICAN HIGHWAY LTD 8%,TJH8.0,JMD,NaN,PREFERENCE,-
65,VICTORIA MUTUAL INVESTMENTS LTD ORDINARY SHARES,VMIL,JMD,FINANCE,ORDINARY,-
66,WIGTON WINDFARM LIMITED ORDINARY SHARES,WIG,JMD,OTHER,ORDINARY,-


In [ ]:
df_listed_jnr

,Name,InstrumentCode,Currency,Sector,Type,Website
0,ACCESS FINANCIAL SERVICES LIMITED,AFS,JMD,FINANCE,ORDINARY,Website
1,AMG PACKAGING & PAPER COMPANY LIMITED,AMG,JMD,MANUFACTURING,ORDINARY,Website
2,BLUE POWER GROUP LIMITED,BPOW,JMD,MANUFACTURING,ORDINARY,-
3,C2W MUSIC LIMITED,MUSIC,JMD,OTHER,ORDINARY,Website
4,CAC 2000 9.5% CUMULATIVE REDEEMABLE PREF SHARES,CAC9.50,JMD,RETAIL,PREFERENCE,-
5,CAC 2000 LIMITED,CAC,JMD,RETAIL,ORDINARY,Website
6,CARGO HANDLERS LIMITED,CHL,JMD,OTHER,ORDINARY,-
7,CARIBBBEAN ASSURANCE BROKERS LIMITED,CABROKERS,JMD,NaN,ORDINARY,-
8,CARIBBEAN CREAM LIMITED,KREMI,JMD,MANUFACTURING,ORDINARY,Website
9,CARIBBEAN FLAVOURS & FRAGRANCES LIMITED,CFF,JMD,MANUFACTURING,ORDINARY,Website


In [ ]:
df_listed[df_listed['Type']=="ORDINARY"].InstrumentCode

0            138SL
2             1834
3              BIL
4              BRG
6              CCC
7              CAR
8             CBNY
11            CPFV
12            EPLY
15    FIRSTROCKJMD
16              GK
17            ITRI
18             JBG
19              JP
26             JSE
32          JMMBGL
35             KEY
36          KPREIT
37              KW
38             MTL
39             MIL
40             MJE
41          MPCCEL
42           NCBFG
43             PAL
44            PJAM
45             PJX
47          PROVEN
48            PULS
49             QWI
50             RJR
51              SJ
52           XFUND
53         SELECTF
54        SELECTMD
55            SALF
56             SGJ
57             SEP
58             SML
59             SIL
60             SVL
61          SCIJMD
62          SCIUSD
63             TJH
65            VMIL
66             WIG
67         WISYNCO
Name: InstrumentCode, dtype: object

In [ ]:
df_listed_jnr[df_listed_jnr['Type']=="ORDINARY"].InstrumentCode

0           AFS
1           AMG
2          BPOW
3         MUSIC
5           CAC
6           CHL
7     CABROKERS
8         KREMI
9           CFF
10          CPJ
11       PURITY
12          DTL
14        DCOVE
15        ELITE
16       EFRESH
17          ECL
18         FTNA
19      FOSRICH
20        GENAC
21        GWEST
22       HONBUN
23      ICREATE
24       INDIES
25          ROC
26          ISP
27         JAMT
28       JETCON
29          KLE
30          KEX
31         LASD
32         LASF
33         LASM
34       LUMBER
35      MAILPAC
36         MEEG
37          MDS
38          PTL
40        SSLVC
41          SOS
42          LAB
43     TROPICAL
44        TTECH
Name: InstrumentCode, dtype: object

In [ ]:
listedComp_lst = df_listed[df_listed['Type']=="ORDINARY"].InstrumentCode.tolist()
listedComp_lst.extend(df_listed_jnr[df_listed_jnr['Type']=="ORDINARY"].InstrumentCode.tolist())

In [ ]:
len(listedComp_lst)

89

In [ ]:
pd.DataFrame(listedComp_lst,columns=["Symbol"]).to_csv('listed_companies.csv')

In [ ]:
postid_lst          = []
title_lst           = []
link_lst            = []
post_date_lst       = []
instument_code_lst  = []
content_lst         = []

for c in listedComp_lst:
  Base_URL  = 'https://www.jamstockex.com/tag/'
  URL = Base_URL + c

  articles_on_page_in_time_range_flag = True
  # Articles are arranged in decending order such that most recent articles are
  # on page 1



  while articles_on_page_in_time_range_flag:
    
    soup = BeautifulSoup(requests.get(URL).content, 'html.parser')

    for article in soup.findAll({'article'}):
      link = article.find('a')
      
      # Print article id
      print(article['id'])
      postid_lst.append(article['id'])

      # Print article title, link
      print(link.string)
      title_lst.append(link.string)
      print(link.get('href'))
      link_lst.append(link.get('href'))

      # Print post date
      post_date = article.find('p', attrs={'class':'text-muted'}).string

      # Print Instrument code
      print(c)
      instument_code_lst.append(c)

      import datetime
      post_date_formatted = post_date.string.replace("Posted: ","")
      post_date_formatted = post_date_formatted.replace(" at", "")
      
      ## Convert date string "post_date" to a format that can be used by strptime
      ## This involves ensuring that the hour characters are padded with 0.

      hour_index = post_date_formatted.find(":") - 2

      if (int(post_date_formatted[hour_index:hour_index+2])) <= 9:
        def insert_zero(string, index):
          return string[:index] + '0' + string[index:]
        post_date_formatted = insert_zero(post_date_formatted,hour_index+1)

      # Print formatted post date
      dt = datetime.datetime.strptime(post_date_formatted, "%B %d, %Y %I:%M %p")
      print(dt)
      post_date_lst.append(dt)

      # Print article content
      content_div = article.find('div', attrs={'class':'entry-content'})
      if content_div != None:
        if content_div.p != None:
          if content_div.p.string != None:
            content_summary = content_div.p.string
          else:
            content_summary = ""
        else:
            content_summary = ""
      else:
        content_summary = ""

      ## Determine if content is a summary by checking for ellipsis string: "[...]"
      ellipsis_string = "[\u2026]"

      if content_summary[-3:] != ellipsis_string:
        print(content_summary)
        content_lst.append(content_summary)
      else:
        print("**Extraction of full text needed**")
        URL_full_article  = link.get('href')
        soup_full_article = BeautifulSoup(requests.get(URL_full_article).content, 'html.parser')

        content_div_full_article=(soup_full_article.find('div', attrs={'class':'entry-content'}))
        
        content_str = ""
        for p in content_div_full_article.findAll({'p'}):
          if p.string != None:
            content_str = content_str + '\n' + p.string
            if len(p.string) >= 100:
              print(p.string)
              break
        content_lst.append(content_str)

      print("")
      if dt <= datetime.datetime(2015,12,31):
        articles_on_page_in_time_range_flag = False
        print("**Exit Scrape, date too old**")

    # Get link for next page
    element = soup.find('a', attrs={'class': "nextpostslink"})
    
    if element != None:
      URL = element.get('href')
    else:
      ## If link for next page can't be found, data extraction complete, so exit loop
      articles_on_page_in_time_range_flag = False

Streaming output truncated to the last 5000 lines.
KEX
2017-09-29 09:26:00
Knutsford Express Annual Report 2017

post-953789
Knutsford Express Services Limited (KEX) Trading in Shares
https://www.jamstockex.com/knutsford-express-services-limited-kex-trading-in-shares/
KEX
2017-08-11 18:03:00
Knutsford Express Services Limited (KEX) has advised that a Director purchased 10,000 KEX shares on August 8, 2017.

post-952059
Knutsford Express Services Limited Audited Financial Statements for the Year Ended May 31, 2017
https://www.jamstockex.com/knutsford-express-services-limited-audited-financial-statements-year-ended-may-31-2017/
KEX
2017-07-28 16:21:00
Knutsford Express Services 2017 FS Finalized JSE

post-951831
Knutsford Express Services Limited (KEX) Declares Dividend
https://www.jamstockex.com/knutsford-express-services-limited-kex-declares-dividend/
KEX
2017-07-26 13:50:00
**Extraction of full text needed**
Knutsford Express Services Limited (KEX) has advised that at the Board of Dire

In [ ]:
df = pd.DataFrame({
    'postid': postid_lst,
    'post_date': post_date_lst,
    'instrument_code':instument_code_lst,
    'title':title_lst,
    'link':link_lst,
    'content': content_lst,
    'content_length': map(len,content_lst),
})

NameError: ignored

In [ ]:
df = pd.read_csv('articles.csv')

In [ ]:
df

,postid,post_date,instrument_code,title,link,content,content_length
0,post-1104646,2020-12-31 16:16:00,138SL,138 Student Living Jamaica Limited (138SL) Aud...,https://www.jamstockex.com/138-student-living-...,138 Student Living Jamaica 2020 – Final 138SL ...,86
1,post-1101933,2020-11-13 18:13:00,138SL,138 Student Living Jamaica Limited (138SL) Una...,https://www.jamstockex.com/138-student-living-...,138 SL FS Sept qtr – 2020 – Final Qtr,37
2,post-1096087,2020-08-14 09:27:00,138SL,138 Student Living Jamaica Limited (138SL) Una...,https://www.jamstockex.com/138-student-living-...,138SL Q3 Financials June 2020,29
3,post-1094046,2020-07-16 16:21:00,138SL,138 Student Living Jamaica Limited (138SL) – I...,https://www.jamstockex.com/138-student-living-...,138 Student Living Jamaica Limited (138SL) – D...,96
4,post-1090194,2020-05-18 15:49:00,138SL,138 Student Living Jamaica Limited (138SL) Una...,https://www.jamstockex.com/138-student-living-...,138SL FS March Qtr 2 2020 138SL Note to JSE,43
...,...,...,...,...,...,...,...
7776,post-937862,2016-07-29 13:10:00,TTECH,tTech Limited (TTECH) – Dividend Consideration,https://www.jamstockex.com/ttech-limited-ttech...,tTech Limited (TTECH) has advised that the Boa...,189
7777,post-486501,2016-05-12 16:35:00,TTECH,"tTech Shareholdings at December 31, 2015",https://www.jamstockex.com/ttech-shareholdings...,"tTech Shareholdings at December 31, 2015",40
7778,post-486495,2016-05-12 16:34:00,TTECH,tTech Limited 2015 Audited Financial Statements,https://www.jamstockex.com/ttech-2015-audited-...,tTech 2015 Audited Financial Statements,39
7779,post-486498,2016-05-12 16:31:00,TTECH,tTech Limited Unaudited Financial Statements a...,https://www.jamstockex.com/ttech-unaudited-fin...,tTech Unaudited Financial Statements at March ...,53


## Generate statistics 

In [ ]:
df.shape

(7781, 7)

In [ ]:
len(df.instrument_code.unique())

88

In [ ]:
df.groupby(['instrument_code']).mean()

,content_length
instrument_code,
138SL,140.542857
1834,192.650000
AFS,175.233333
AMG,132.400000
BIL,207.069231
...,...
TTECH,119.000000
VMIL,164.642857
WIG,137.333333


In [ ]:
df.groupby(['instrument_code']).count()

,postid,post_date,title,link,content,content_length
instrument_code,,,,,,
138SL,70,70,70,70,69,70
1834,80,80,80,80,79,80
AFS,90,90,90,90,89,90
AMG,70,70,70,70,70,70
BIL,130,130,130,130,130,130
...,...,...,...,...,...,...
TTECH,69,69,69,69,69,69
VMIL,70,70,70,70,70,70
WIG,12,12,12,12,12,12


In [ ]:
df.describe()

,content_length
count,7781.000000
mean,155.740008
std,105.892287
min,0.000000
25%,94.000000
50%,131.000000
75%,209.000000
max,1569.000000


In [ ]:
df.to_csv('articles.csv',index=False)

# Generate statistics on trading information

In [ ]:
df_trading = pd.read_csv('price-history.csv')

In [ ]:
df_trading

,Symbol,Date,52_Week_High,52_Week_Low,Previous_Year_Div,Current_Year_Div,Volume_non_block,Today_High,Today_Low,Last_Traded,Close_Price,Price_Change,Closing_Bid,Closing_Ask
0,AFS,4/1/2016,19.00,9.20,0.5721,NaN,8510.0,16.00,16.00,16.00,16.000000,0.00,16.000,17.22
1,BIL,4/1/2016,4.50,1.95,0.0700,NaN,NaN,NaN,NaN,3.80,3.800000,0.00,3.400,3.80
2,BRG,4/1/2016,3.80,1.55,0.1200,NaN,85100.0,3.20,3.15,3.15,3.190000,-0.01,3.120,3.20
3,BPOW,4/1/2016,14.00,6.30,0.1500,NaN,NaN,NaN,NaN,13.10,13.100000,0.00,10.050,12.90
4,CWJ,4/1/2016,2.00,0.39,NaN,NaN,212690.0,1.40,1.32,1.38,1.370000,-0.01,1.380,1.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133965,TOKI,31/12/2020,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.01,0.010000,0.00,0.012,0.00
133966,CWJDEFERREDA,31/12/2020,4.00,1.50,NaN,NaN,497317.0,2.10,1.99,2.10,2.020102,0.17,1.700,2.10
133967,LUMBER,31/12/2020,1.88,0.95,NaN,NaN,224630.0,1.58,1.45,1.56,1.543207,0.02,1.470,1.55
133968,TROPICAL,31/12/2020,1.35,1.01,NaN,NaN,306400.0,1.20,1.13,1.20,1.190992,0.01,1.170,1.22


In [ ]:
df_trading_filtered = df_trading[df_trading['Symbol'].isin(listedComp_lst)]

In [ ]:
df_trading_filtered

,Symbol,Date,52_Week_High,52_Week_Low,Previous_Year_Div,Current_Year_Div,Volume_non_block,Today_High,Today_Low,Last_Traded,Close_Price,Price_Change,Closing_Bid,Closing_Ask
0,AFS,4/1/2016,19.00,9.20,0.5721,NaN,8510.0,16.00,16.00,16.00,16.000000,0.00,16.00,17.22
1,BIL,4/1/2016,4.50,1.95,0.0700,NaN,NaN,NaN,NaN,3.80,3.800000,0.00,3.40,3.80
2,BRG,4/1/2016,3.80,1.55,0.1200,NaN,85100.0,3.20,3.15,3.15,3.190000,-0.01,3.12,3.20
3,BPOW,4/1/2016,14.00,6.30,0.1500,NaN,NaN,NaN,NaN,13.10,13.100000,0.00,10.05,12.90
7,CHL,4/1/2016,37.50,16.00,2.3500,NaN,NaN,NaN,NaN,37.50,37.500000,0.00,37.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133960,FIRSTROCKJMD,31/12/2020,18.49,8.50,NaN,NaN,45211.0,12.70,12.48,12.64,12.552716,-0.09,12.35,12.70
133962,CABROKERS,31/12/2020,2.85,1.60,NaN,NaN,1300.0,1.89,1.75,1.89,1.884615,-0.01,1.75,1.89
133963,TJH,31/12/2020,1.41,1.00,NaN,NaN,10197113.0,1.36,1.30,1.35,1.326428,0.00,1.33,1.35
133967,LUMBER,31/12/2020,1.88,0.95,NaN,NaN,224630.0,1.58,1.45,1.56,1.543207,0.02,1.47,1.55


In [ ]:
df_trading_filtered.groupby(['Symbol']).mean()

,52_Week_High,52_Week_Low,Previous_Year_Div,Current_Year_Div,Volume_non_block,Today_High,Today_Low,Last_Traded,Close_Price,Price_Change,Closing_Bid,Closing_Ask
Symbol,,,,,,,,,,,,
138SL,6.521710,3.398465,NaN,NaN,1.932724e+05,3.086730,2.995701,4.724678,4.787286,0.000239,4.369177,4.974070
1834,1.698186,0.910064,0.095375,0.081269,1.860156e+05,1.042168,1.011354,1.197144,1.221541,-0.000167,1.163137,1.270287
AFS,44.405322,22.791305,0.768973,0.455271,1.062513e+04,24.224919,23.661525,33.358425,34.408079,0.014063,33.444223,37.135056
AMG,12.140899,2.444735,0.326596,0.416160,1.332645e+05,4.590503,4.425535,5.662538,5.705274,0.011225,5.529378,6.163728
BIL,41.314495,13.167733,0.558790,0.561566,1.807619e+05,35.246006,33.994787,30.219189,30.227441,0.061226,30.226163,30.932926
...,...,...,...,...,...,...,...,...,...,...,...,...
TTECH,7.892568,4.169641,0.039782,0.039969,1.929260e+04,4.136204,4.016379,5.580463,5.670429,0.001243,5.416637,6.028630
VMIL,8.929089,3.614016,0.190000,0.041765,2.626536e+05,5.836222,5.578970,5.715020,5.697743,0.001500,5.648666,5.788666
WIG,1.113309,0.550809,NaN,NaN,1.578071e+07,0.847819,0.797868,0.822475,0.819251,0.000646,0.810858,0.832230


In [ ]:
df_trading_filtered.groupby(['Symbol']).count()

,Date,52_Week_High,52_Week_Low,Previous_Year_Div,Current_Year_Div,Volume_non_block,Today_High,Today_Low,Last_Traded,Close_Price,Price_Change,Closing_Bid,Closing_Ask
Symbol,,,,,,,,,,,,,
138SL,1257,1257,1257,0,0,1061,942,942,1257,1257,1257,1203,1150
1834,1257,1257,1257,973,268,1194,1130,1130,1257,1257,1257,1240,1256
AFS,1257,1257,1257,1162,850,1058,931,931,1257,1257,1257,1255,1151
AMG,1257,1257,1257,752,250,1142,1093,1093,1257,1257,1257,1254,1250
BIL,1257,1257,1257,1162,719,1113,1034,1034,1257,1257,1257,1238,1251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTECH,1255,1254,1254,275,81,1154,1088,1088,1253,1255,1255,1249,1248
VMIL,757,757,757,162,272,757,757,757,757,757,757,757,757
WIG,408,408,408,0,0,408,408,408,408,408,408,408,408


In [ ]:
grouped = df_trading_filtered.groupby(['Symbol']).count().reset_index()

In [ ]:
grouped.sort_values(['Last_Traded'],ascending=False)

,Symbol,Date,52_Week_High,52_Week_Low,Previous_Year_Div,Current_Year_Div,Volume_non_block,Today_High,Today_Low,Last_Traded,Close_Price,Price_Change,Closing_Bid,Closing_Ask
0,138SL,1257,1257,1257,0,0,1061,942,942,1257,1257,1257,1203,1150
47,LASM,1257,1257,1257,1162,349,1257,1244,1244,1257,1257,1257,1255,1254
32,JAMT,1257,1257,1257,410,267,1160,1138,1138,1257,1257,1257,1254,1253
33,JBG,1257,1257,1257,1162,691,1250,1246,1246,1257,1257,1257,1256,1255
35,JMMBGL,1257,1257,1257,1162,432,1248,1248,1248,1257,1257,1257,1256,1256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,SELECTMD,262,262,262,0,0,262,262,262,262,262,262,262,262
22,FIRSTROCKJMD,218,218,218,0,0,218,218,218,218,218,218,218,218
7,CABROKERS,209,209,209,0,0,209,209,209,209,209,209,209,209
81,TJH,198,198,198,0,0,198,198,198,198,198,198,198,198


In [ ]:
grouped.to_csv('trading_count.csv')